# $$ \text{ Identifying Duplicate Listings }$$ 
><br/>The goal of this test is to propose a listings duplicate detection method. Ou datasets of listing contains 1427 listings each listing provides the following information:<br/>
>* listing_id
>* surface_m2
>* description 
>* current_price
>* city
>* transaction_type
>* item_type
>* room_count
>* floor
>* floor_count


In [2]:
import csv
import numpy as np
import pandas as pd
import itertools, functools
import copy
from itertools import combinations

# $ \text{ 1. Loading Data} $

>* df_listing: is the dataset containning 1427 listings
>* df_dlisting: is a dataset containing the pairs of duplicate listings<br/> 
<br/>
We flatten df_dlisting in order to have groups of duplicates instead of pairs, we also replace Nan value in df_listing with the value -1<br/><br/>


In [3]:
df_dlisting = pd.read_csv("./ml_test_dataset/duplicate_listing.csv")
df_listing = pd.read_csv("./ml_test_dataset/listing.csv")

display(df_listing)
display(df_dlisting)

,listing_id,surface_m2,description,room_count,floor,floor_count,current_price,city,city_zip,transaction_type,item_type
0,121205222,57,*** CONFLANS SAINTE HONORINE - proche COMMERCE...,4.0,3.0,NaN,195000,Conflans-Sainte-Honorine,78700,sell,apartment
1,110085329,48,** AGENCE WEELODGE ** Dans le secteur de FIN D...,2.0,NaN,NaN,157500,Conflans-Sainte-Honorine,78700,sell,apartment
2,122402493,53,*** CONFLANS SAINTE HONORINE - IDEAL 1ER ACHAT...,3.0,2.0,NaN,142000,Conflans-Sainte-Honorine,78700,sell,apartment
3,117274348,67,** AGENCE WEELODGE ** PROCHE GARE SNCF et CENT...,4.0,4.0,NaN,185000,Conflans-Sainte-Honorine,78700,sell,apartment
4,123155187,75,** AGENCE WEELODGE CONFLANS ** A 10MIN DE LA G...,4.0,3.0,4.0,189900,Conflans-Sainte-Honorine,78700,sell,apartment
...,...,...,...,...,...,...,...,...,...,...,...
1423,117949689,54,*** CONFLANS SAINTE HONORINE *** Situé à PROXI...,3.0,1.0,NaN,179500,Conflans-Sainte-Honorine,78700,sell,apartment
1424,118800823,62,** EXCLUSIVITÉ ** CONFLANS: Proche GARES RER/ ...,3.0,0.0,4.0,198000,Conflans-Sainte-Honorine,78700,sell,apartment
1425,118800825,55,CONFLANS: EMPLACEMENT IDÉAL ! Situé en plein c...,3.0,1.0,1.0,179500,Conflans-Sainte-Honorine,78700,sell,apartment
1426,115945848,67,CONFLANS ***EXCLUSIVITÉ*** À 800m de la GARE S...,4.0,0.0,4.0,169000,Conflans-Sainte-Honorine,78700,sell,apartment


,listing_id_1,listing_id_2
0,64721495,64728971
1,64721495,65459581
2,64728971,65459581
3,98430083,98429480
4,119279327,116630374
...,...,...
3579,121226460,121615906
3580,67292960,69521695
3581,67292960,68728687
3582,119227626,120836693


### $ \text{ 1.2 Fill Nan Values} $


> <br/> Only the columns **room-count**, **floor** and **floor_count** have nan values.<br/><br/>


In [4]:
print(df_listing.isna().sum())
# Sort columns acoording to the number of Nans, this order will be used to group the listings 
sorted_columns = df_listing.isna().sum().sort_values(ascending = True).index
# Fill Nan values with -1
df_listing = df_listing.fillna(-1)

listing_id            0
surface_m2            0
description           0
room_count            3
floor               151
floor_count         474
current_price         0
city                  0
city_zip              0
transaction_type      0
item_type             0
dtype: int64



### $ \text{ 1.3 Form groups from pairs of duplicates} $

In [5]:
# form groups of duplicates from pairs 

def partition(pred, iterable):
    t1, t2 = itertools.tee(iterable)
    return itertools.filterfalse(pred, t1), filter(pred, t2)

groups = []
list_ids = list(df_listing['listing_id'])
for i in range(len(df_dlisting)):
    a, b = df_dlisting.loc[i]
    unrelated, related = partition(lambda group: any(aa == a or bb == b or aa == b or bb == a for aa, bb in group), groups)
    groups = [*unrelated, sum(related, [(a, b)])]
flaten_groups = []
for grp in groups:
    g = set([j for tuple in grp for j in tuple if j in list_ids])
    if len(g)>1:
        flaten_groups.append(list(g))
print(f"We have {len(flaten_groups)} groups of duplicates")

We have 277 groups of duplicates


# $ \text{2. Detect duplicates}$

> <br/>In this section we will first detect the duplicates that match at 100\% using pandas library, then we form groups from the remaining listings according to differents columns. Once the small groups are formed we will run a Deep Learning model to estimate the similarity between listings of the same group. <br/><br/>

### $ \text{2.1 Detect 100\% matching duplicates}$

> <br/>To Better match between the **descriptions** we will run a small text preprocessing to remove spaces and special caracters<br/><br/>

In [6]:
# Preprocessing of description
import re
def text_processing(s):
    if s != None:
        s = s.lower()
        s = s.replace('<br />', '')
        s = re.sub('[.,;=:!@#$+<>-]', ' ', s)
        s = " ".join(s.split())
    return s
df_listing['description'] = df_listing['description'].map(lambda x: text_processing(x))


In [7]:
# Remove duplicates and save the ids of duplicated listings

columns = list(df_listing.columns)
df_listing = df_listing.fillna(-1)
columns.remove("listing_id")
duplicates = df_listing.duplicated(subset=columns, keep=False)

direct_matches = df_listing.loc[duplicates, :]
# Groupby columns in order to get the duplicates in groups
direct_matches_gb = direct_matches.groupby(by=columns, dropna=False)
predicted_pairs = []
for key, item in direct_matches_gb:
    predicted_pairs.append(list(direct_matches_gb.get_group(key)['listing_id']))

# Remove Duplicates and keeping only the first occurence
new_df_listing = df_listing.drop_duplicates(subset=columns, keep='first')
print(f'{len(df_listing)-len(new_df_listing)} removed listings')

421 removed listings


### $ \text{2.2 Group by columns}$

> <br/>After removing the obvious duplicates, we will groupby our dataset on the columns `['city', 'city_zip', 'item_type', 'room_count', 'floor', 'floor_count']`. We first start to group by the columns that don't have nan value. As for the columns that have nan values (previously replaced with -1), we remove the sub-group formed with the `key=-1` and add it to the other sub-group; since the value is missing, the listing could belong to any sub-group.<br/><br/>
The column `['current_price']` is removed since the price of duplicates can be different. For the column `['surface_m2']` the duplicates can have a diffent surfface but within an interval of more or less 3 $m^2$<br/><br/>


In [8]:
rm_col = ['surface_m2', 'current_price', 'listing_id', 'description']
columns = [c for c in sorted_columns if c not in rm_col]
print(columns)

['city', 'city_zip', 'transaction_type', 'item_type', 'room_count', 'floor', 'floor_count']


In [9]:
df = copy.deepcopy(new_df_listing)
grps = [df]
 
for c in columns:
    new_grps = []
    for g in grps:
            # Group By column c
            group_by_df = g.groupby(by=c, dropna=False)
            nan_group = None
            keys = list(group_by_df.agg(lambda x: list(x)).index)
            if -1 in keys:
                nan_group = group_by_df.get_group(-1)
            for key, item in group_by_df:
                if key!=-1:
                    if nan_group is None:
                        new_grps.append(group_by_df.get_group(key))
                    else:
                        # concatenate with nan Group
                        new_grps.append(pd.concat([group_by_df.get_group(key), nan_group]))
                
    grps = new_grps

In [10]:
def process_surface(group, df, v=3):
    """
    Check if the surfaces of the group are in the same interval
    """
    df_g = df.loc[df['listing_id'].isin(group)]
    surfaces = pd.Series(df_g['surface_m2']).value_counts()
    if len(surfaces)>1:
        new_g = []
        max_surface = surfaces.index[0]
        for id in group: 
            id_surface = list(df_g.loc[df_g['listing_id']==id]['surface_m2'])[0]
            if abs(max_surface-id_surface) <= v:
                new_g.append(id)
        return new_g
    else:
        return list(g)

### $ \text{2.3 Sentence-Bert}$


> <br/>The idea of Sentence-BERT is to modify the architecture of BERT into a Siamese network.
The two sentences of a pair go through a BERT followed by a pooling operation to obtain the two
embeddings u and v. Several methods can be used for the pooling step, although
the researchers found that mean pooling worked better.<br/><br/>
After the pooling is done, we have 2 embeddings: 1 for sentence A and 1 for sentence B. At inference
the two embeddings are then compared using a cosine similarity function, which will output a similarity score for the two sentences. <br/><br/>

![sentence-Bert](sbert.png)


In [11]:
from sentence_transformers import SentenceTransformer, util

# Get pretrained Model
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

for g in grps:
    g_embeddings = model.encode(list(g['description']))
    for e in g_embeddings:
            ids = np.array(g['listing_id'])
            results = np.array((util.dot_score(e, g_embeddings)>0.9).squeeze(0))
            duplicates = list(ids[results])
            # ADD to duplicates
            if len(duplicates)>1:
                # Check if the surfaces are correct
                duplicates = process_surface(duplicates, new_df_listing)
                for pairs in predicted_pairs:
                    if any(item in duplicates for item in pairs):
                        # Remove pairs from duplicates
                        predicted_pairs.remove(pairs)
                        duplicates = list(set(list(duplicates)+list(pairs)))
                predicted_pairs.append(duplicates)

### $ \text{2.4 Results}$

In [86]:
# Precision and Recall considering individual listings (not pairs)

groud_truth = set(sum([list(l) for l in flaten_groups if len(l)>1], []))
prediction = set(sum(predicted_pairs, []))
intersection = groud_truth.intersection(prediction)
print('TP (Number of duplicates that have been predicted):',len(intersection))
print('TN (Number of duplicates that have not been predicted):',(len(groud_truth) - len(intersection)))
print('FP (Number of wrong prediction):',(len(prediction) - len(groud_truth.intersection(prediction))))
print('Precision: {:.2f}'.format(len(intersection)/len(prediction)))
print('Recall: {:.2f}'.format(len(intersection)/len(groud_truth)))

TP (Number of duplicates that have been predicted): 1078
TN (Number of duplicates that have not been predicted): 152
FP (Number of wrong prediction): 12
Precision: 0.99
Recall: 0.88


In [85]:
# Precision and Recall considering pairs/couples of listings

all_predicted_pairs = set(sum([list(combinations(sorted(set(pair)), 2)) for pair  in predicted_pairs], []))
all_ground_truth_pairs = set(sum([list(combinations(sorted(set(pair)), 2)) for pair in flaten_groups ], []))
intersection = all_ground_truth_pairs.intersection(all_predicted_pairs)

print('Precision: {:.2f}'.format(len(intersection)/len(all_predicted_pairs)))
print('Recall: {:.2f}'.format(len(intersection)/len(all_ground_truth_pairs)))

Precision: 0.89
Recall: 0.68


> Some errors in the df_dlisting dataset (duplicates pairs dataset) : 74 couples have different **floor** numbers but labeled as duplicates 

### $ \text{2.5 Ideas to improve the solution}$

* Finetune the sentence-Bert model on our data to make it more specific to our task.

* Use a model to detect [
Image Duplicates & Near Duplicates.](https://github.com/UKPLab/sentence-transformers/blob/master/examples/applications/image-search/Image_Duplicates.ipynb)

* Make sure to have correctly labeled datasets.